In [75]:
from sec_processing.utils import *
import settings
from typing import List, Dict

In [76]:
def fetch_report_to_df(ticker: str, report_type: str) -> pd.DataFrame:
    """Given a ticker and valid report type, this function returns the report for the latest period"""
    ticker = ticker.upper()
    report_type = report_type.lower()

    valid_report_types = ["balance_sheet", "income_statement", "cash_flow_statement"]
    if report_type not in valid_report_types:
        raise ValueError(f"{report_type} is not a valid report type")

    try:
        acc = get_filtered_filings(ticker, form_type="10-Q", just_accession_numbers=True)
        acc_num = acc.iloc[0].replace('-', '')
    except IndexError:
        raise ValueError(f"There was a problem getting filings for {ticker}")

    try:
        statement = process_one_statement(ticker, acc_num, report_type)
    except ValueError:
        print("stop program")
        raise ValueError(f"There was a problem getting filings for {ticker}")

    label_dict = get_label_dictionary(ticker, headers)
    statement_df = rename_statement(statement, label_dict)
    #settings.logger.debug("statement_df: {}".format(statement_df))
    return statement_df


def extract_tax_rate(df_: pd.DataFrame) -> float:
    ebt_ = df_.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest']
    tax_expense_ = df_.loc['Income Tax Expense (Benefit)']
    tx_raw_ = (tax_expense_ / ebt_)
    tc_ = tx_raw_.apply(lambda x: max(0, x))
    return tc_


In [ ]:
ticker = "XOM"
report_type = "income_statement"
df = fetch_report_to_df(ticker, report_type)

# Extract EBT and Income Tax rows
ebt = df.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest']
tax_expense = df.loc['Income Tax Expense (Benefit)']

# Calculate Effective Tax Rate
tc = tax_expense / ebt

# Optional: handle negative EBT to avoid nonsensical negative tax rates
tc_clean = tc.apply(lambda x: max(0, x))

print("Effective Tax Rate (Tc) per report date:")
print(tc_clean)


In [89]:


tickers_list = ["AAPL", "MSFT", "JNJ", "JPM", "XOM", "PG", "NVDA", "UNH", "HD", "DIS"]


def workflow(tickers_: List[str]) -> list[Dict]:
    data = []
    report_type_w = "income_statement"
    for ticker_ in tickers_:
        current_ticker = ticker_  #tickers_list[ticker_]
        try:
            df_w = fetch_report_to_df(current_ticker, report_type_w)
            tc_w = extract_tax_rate(df_w)
            data.append({"ticker": current_ticker, "tc": tc_w})
            settings.logger.info(f"Processed {current_ticker}")
        except Exception as e:
            settings.logger.error(f"Error processing {current_ticker}: {e}")

    return data


def convert_and_group(data_list):
    # Initialize empty list to collect records
    records = []

    # Loop through each dictionary in the input list
    for entry in data_list:
        ticker = entry['ticker']
        tc_series = entry['tc']

        # Convert each series to dataframe and reset index for date-value structure
        df = tc_series.reset_index()
        df.columns = ['date', 'value']
        df['ticker'] = ticker

        # Append to records
        records.append(df)

    # Concatenate all records into one dataframe
    final_df = pd.concat(records, ignore_index=True)

    # Reorder columns
    final_df = final_df[['ticker', 'date', 'value']]
    # Group by ticker and calculate average
    grouped_df = final_df.groupby('ticker')['value'].mean().reset_index()

    return final_df, grouped_df



In [95]:

tickers = ["MSFT", "JNJ", "HD", "GOOGL", "TSLA", "HD", "JNJ"]
tickers_tc = workflow(tickers)
summarized_tc_df, stats_per_symbol = convert_and_group(tickers_tc)
print(summarized_tc_df.sort_values(by=['value'], ascending=False))



15:35:34.188 Processed MSFT
15:35:35.847 Processed JNJ
15:35:37.671 Processed HD
15:35:39.484 Processed GOOGL
15:35:41.275 Processed TSLA
15:35:43.015 Processed HD
15:35:44.762 Processed JNJ
[{'ticker': 'MSFT', 'tc': 2025-03-31    0.176977
2024-03-31    0.179145
2025-03-31    0.180330
2024-03-31    0.179259
dtype: float64}, {'ticker': 'JNJ', 'tc': 2025-03-30    0.193089
2024-03-31    0.123586
dtype: float64}, {'ticker': 'HD', 'tc': 2025-05-04    0.244166
2024-04-28    0.225973
dtype: float64}, {'ticker': 'GOOGL', 'tc': 2025-03-31    0.173467
2024-03-31    0.164330
dtype: float64}, {'ticker': 'TSLA', 'tc': 2025-03-31    0.286927
2024-03-31    0.255826
dtype: float64}, {'ticker': 'HD', 'tc': 2025-05-04    0.244166
2024-04-28    0.225973
dtype: float64}, {'ticker': 'JNJ', 'tc': 2025-03-30    0.193089
2024-03-31    0.123586
dtype: float64}]


In [92]:
len(comparisson)

5

Full DataFrame:
   ticker       date     value
0    MSFT 2025-03-31  0.176977
1    MSFT 2024-03-31  0.179145
2    MSFT 2025-03-31  0.180330
3    MSFT 2024-03-31  0.179259
4     JNJ 2025-03-30  0.193089
5     JNJ 2024-03-31  0.123586
6      HD 2025-05-04  0.244166
7      HD 2024-04-28  0.225973
8   GOOGL 2025-03-31  0.173467
9   GOOGL 2024-03-31  0.164330
10   TSLA 2025-03-31  0.286927
11   TSLA 2024-03-31  0.255826
12     HD 2025-05-04  0.244166
13     HD 2024-04-28  0.225973
14    JNJ 2025-03-30  0.193089
15    JNJ 2024-03-31  0.123586

Average by Ticker:
  ticker     value
0  GOOGL  0.168898
1     HD  0.235069
2    JNJ  0.158338
3   MSFT  0.178928
4   TSLA  0.271377


(   ticker       date     value
 0    MSFT 2025-03-31  0.176977
 1    MSFT 2024-03-31  0.179145
 2    MSFT 2025-03-31  0.180330
 3    MSFT 2024-03-31  0.179259
 4     JNJ 2025-03-30  0.193089
 5     JNJ 2024-03-31  0.123586
 6      HD 2025-05-04  0.244166
 7      HD 2024-04-28  0.225973
 8   GOOGL 2025-03-31  0.173467
 9   GOOGL 2024-03-31  0.164330
 10   TSLA 2025-03-31  0.286927
 11   TSLA 2024-03-31  0.255826
 12     HD 2025-05-04  0.244166
 13     HD 2024-04-28  0.225973
 14    JNJ 2025-03-30  0.193089
 15    JNJ 2024-03-31  0.123586,
   ticker     value
 0  GOOGL  0.168898
 1     HD  0.235069
 2    JNJ  0.158338
 3   MSFT  0.178928
 4   TSLA  0.271377)

In [99]:
final_df, stats_per_symbol = convert_and_group(comparisson)

Full DataFrame:
   ticker       date     value
0    MSFT 2025-03-31  0.176977
1    MSFT 2024-03-31  0.179145
2    MSFT 2025-03-31  0.180330
3    MSFT 2024-03-31  0.179259
4     JNJ 2025-03-30  0.193089
5     JNJ 2024-03-31  0.123586
6      HD 2025-05-04  0.244166
7      HD 2024-04-28  0.225973
8   GOOGL 2025-03-31  0.173467
9   GOOGL 2024-03-31  0.164330
10   TSLA 2025-03-31  0.286927
11   TSLA 2024-03-31  0.255826
12     HD 2025-05-04  0.244166
13     HD 2024-04-28  0.225973
14    JNJ 2025-03-30  0.193089
15    JNJ 2024-03-31  0.123586

Average by Ticker:
  ticker     value
0  GOOGL  0.168898
1     HD  0.235069
2    JNJ  0.158338
3   MSFT  0.178928
4   TSLA  0.271377


In [101]:
print(final_df.head())

  ticker       date     value
0   MSFT 2025-03-31  0.176977
1   MSFT 2024-03-31  0.179145
2   MSFT 2025-03-31  0.180330
3   MSFT 2024-03-31  0.179259
4    JNJ 2025-03-30  0.193089


  ticker     value
4   TSLA  0.271377
1     HD  0.235069
3   MSFT  0.178928
0  GOOGL  0.168898
2    JNJ  0.158338
